In [1]:
%%capture
import mne
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.signal import welch


In [2]:
%%capture
#Convert to DataFrame
def convert_bdf_to_dataframe(bdf_filename):
    raw_data = mne.io.read_raw_bdf(bdf_filename, preload=True)
    eeg_data_raw = raw_data.get_data()
    channel_names = raw_data.ch_names
    time_index = raw_data.times
    eeg_data = pd.DataFrame(data=eeg_data_raw.T, columns=channel_names, index=time_index)
    col_names = ['O1', 'O2', 'F3', 'F4', 'C3' ,'C4' , 'Fp1', 'Fp2']
    eeg_data = eeg_data[col_names]
    segment_size = 1024
    num_segments = len(eeg_data) // segment_size
    reduced_df = pd.DataFrame(columns=eeg_data.columns)
    for i in range(num_segments):
        start_idx = i * segment_size
        end_idx = (i + 1) * segment_size
        segment_data = eeg_data.iloc[start_idx:end_idx]    
        mean_values = segment_data.mean()
        reduced_df = reduced_df.append(mean_values, ignore_index=True)

    return reduced_df

In [3]:
%%capture
# Define a function to extract labels from filenames
def extract_label_from_filename(filename):
    # Assuming filenames are in the format: subject_task.bdf
    task = filename.split('_')[1].split('.')[0]
    sub = filename.split('_')[0]
    return task, sub

In [7]:
# Example code to create a spectrogram image from each EEG channel
def eeg_channel_to_spectrogram(channel_data, ch_idx):
    plt.specgram(channel_data, NFFT=256, Fs=1000, noverlap=128)
    plt.axis('off')  # Turn off axis
#     plt.savefig(f'{ch_idx}_channel_spectrogram.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    image = Image.open(f'{ch_idx}_channel_spectrogram.png')
    path = f'{ch_idx}_channel_spectrogram.png'
    return image, path

In [13]:
%%capture
from PIL import Image
# Load and preprocess the data
data_dir = './Dset'  # Replace with the path to your .bdf data directory
eeg_channel_images = []

subject_data = {}

output_dir = './output_images'
Med1_dir = './output_images/med1breathe'
MedN_dir = './output_images/medNormal'
think1_dir = './output_images/think1'
think2_dir = './output_images/think2'
dirs = [thin2_dir, think1_dir, MedN_dir, output_dir, Med1_dir]
for i in dirs:
    os.makedirs(i, exist_ok=True)

# Loop through all files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.bdf'):
        # Convert the .bdf file to a DataFrame
        eeg_data = convert_bdf_to_dataframe(os.path.join(data_dir, filename))
        
        # Extract labels from filenames
        label, sub = extract_label_from_filename(filename)
        
        if sub not in subject_data:
            subject_data[sub] = []
            
        for channel_idx in range(eeg_data.shape[1]):
            channel_data = eeg_data.iloc[:,channel_idx]
            channel_image, path = eeg_channel_to_spectrogram(channel_data, eeg_data.columns[channel_idx])
            image_filename = f'{sub}_{label}_{path}'
            if label == 'med1breath':
                image_path = os.path.join(Med1_dir, image_filename)
                channel_image.save(image_path)
                eeg_channel_images.append(image_filename)
            elif label == 'medNormal':
                image_path = os.path.join(MedN_dir, image_filename)
                channel_image.save(image_path)
                eeg_channel_images.append(image_filename)
            elif label == 'think1':
                image_path = os.path.join(think1_dir, image_filename)
                channel_image.save(image_path)
                eeg_channel_images.append(image_filename)
            else:
                image_path = os.path.join(think2_dir, image_filename)
                channel_image.save(image_path)
                eeg_channel_images.append(image_filename)
                
            subject_data[sub].append(channel_data)
            

In [9]:
len(eeg_channel_images)

64